In [1]:
import json
from src.generator import GeneratorFactory
from src.prompts import CfgNetPromptSettings
from src.util import transform
from dotenv import load_dotenv
import os

load_dotenv(dotenv_path=".env")

print(f"Base URL: {os.getenv("BASE_URL")}")
print(f"Proxy Server Key: {os.getenv("PROXY_SERVER_API_KEY")}")


# TODO: define genrators
generator = GeneratorFactory().get_generator(model_name="gpt-4o-2024-11-20", temperature=0.7)

with open("data/evaluation/test_dependencies_retrieval.json", "r", encoding="utf-8") as src:
    data = json.load(src)

for sample in data:
    dependency = transform(entry=sample)

    # create prompt parts
    system_str = CfgNetPromptSettings.get_system_str(dependency=dependency)
    task_str = CfgNetPromptSettings.get_task_str(dependency=dependency)
    context_str = "\n\n".join([x["text"] for x in sample["context"]])
    format_str = CfgNetPromptSettings.get_format_prompt()

    query_str = CfgNetPromptSettings.query_prompt.format(
        context_str=context_str,
        task_str=task_str,
        format_str=format_str
    )

    messages = [
        {"role": "system", "content": system_str},
        {"role": "user", "content": query_str }
    ]

    responses = generator.generate(messages=messages)

    print(responses)

    break



/Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Base URL: http://172.26.92.115
Proxy Server Key: 0ad58c3e-b9f4-4913-b8c9-7cd8e2fec7ab
GPT (gpt-4o-2024-11-20) generator initialized.
Start generation using gpt-4o-2024-11-20


[02/02/25 14:03:36] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=730401;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=337355;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Start generation using gpt-4o-2024-11-20


[02/02/25 14:03:41] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=604796;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=163080;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Start generation using gpt-4o-2024-11-20


[02/02/25 14:03:45] INFO     HTTP Request: POST http://172.26.92.115/chat/completions "HTTP/1.1 200 ]8;id=915547;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py\_client.py]8;;\:]8;id=87695;file:///Users/sebastiansimon/GitHub/rag-config-val/env/lib/python3.12/site-packages/httpx/_client.py#1025\1025]8;;\
                             OK"                                                                                   

Add response to answer pool.
Get dominant response.
{'plan': "1. Identify the configuration options to analyze: 'project.modelVersion' in 'apollo-audit/apollo-audit-api/pom.xml' and 'project.modelVersion' in 'apollo-biz/pom.xml', both of type VERSION_NUMBER with the value 'modelVersion:4.0.0'.\n2. Recognize that both configuration options belong to Maven's POM (Project Object Model) structure, which uses 'modelVersion' to define the version of the POM model being used.\n3. Determine the purpose of 'modelVersion' in Maven. It specifies the version of the POM model and must be set to a specific value (usually '4.0.0') for Maven to correctly parse and build the project.\n4. Evaluate if the value-equality of 'modelVersion' in these files is an accidental coincidence or a necessary dependency.\n5. Conclude whether the configuration options depend on each other by analyzing Maven's requirements for 'modelVersion'.", 'rationale': "In Maven, the 'modelVersion' configuration option defines the 

In [2]:
from src.util import get_dominat_response

responses = [
    {
    "plan": "1. Identify the configuration options in question: both are 'project.modelVersion' of type VERSION_NUMBER with value 4.0.0.\n2. Check if these configurations belong to different Maven modules (apollo-audit/apollo-audit-api and apollo-biz).\n3. Determine if there's a parent-child relationship between the POM files or if they are independent.\n4. Assess whether Maven requires modelVersion to be consistent across modules in a multi-module project.\n5. Conclude based on Maven's requirements for modelVersion consistency.",
    "rationale": "In Maven, each module's pom.xml must have the same modelVersion as specified in the parent POM if it's part of a multi-module project. If these modules share a common parent with a defined modelVersion, then their individual 'project.modelVersion' values must match to maintain consistency and avoid configuration errors.",
    "uncertainty": 8,
    "isDependency": True
    },
    {
    "plan": "1. Identify both configuration options in their respective POM files.\n2. Check if they are of the same type (VERSION_NUMBER) and have the same value (4.0.0).\n3. Determine if Maven requires these versions to be identical for proper functionality across dependent projects.\n4. Conclude based on Maven's handling of modelVersion in multi-module projects.",
    "rationale": "In Maven, the modelVersion typically refers to the version of the POM structure and is usually consistent across all modules within a multi-module project to ensure compatibility. If apollo-audit-api and apollo-biz are part of the same parent project or have interdependencies, having the same modelVersion ensures that Maven can correctly process and build them together without conflicts.",
    "uncertainty": 8,
    "isDependency": True
    },
    {
    "plan": "1. Identify the two configuration options in question: project.modelVersion in apollo-audit/apollo-audit-api/pom.xml with value 4.0.0 and project.modelVersion in apollo-biz/pom.xml also with value 4.0.0.\n2. Understand that both are Maven POM files, which use modelVersion to specify the version of the POM format they conform to.\n3. Recognize that modelVersion is a standard element in Maven POMs and typically refers to the Project Object Model version (e.g., 4.0.0).\n4. Note that having the same value doesn't inherently create a dependency; it's about whether their values must match for proper functionality.\n5. Consider whether these two projects are part of the same multi-module project or independent.\n6. If they're modules in a parent POM, the modelVersion should align with the parent's modelVersion to maintain consistency and avoid errors during build processes.\n7. However, if they are separate projects without a shared parent, their modelVersions can differ without causing dependency issues because each project is built independently.\n8. Without explicit information about a parent-child relationship or inter-dependency between these two POMs, it's reasonable to conclude that their modelVersion values don't depend on each other.\n9. Therefore, the same value might be coincidental and not indicative of an actual dependency.",
    "rationale": "The project.modelVersion in both POM files refers to the Maven POM version specification. While they share the same value (4.0.0), this does not imply a dependency unless one is a parent or module of the other, which isn't indicated here. Maven projects can independently have the same modelVersion without being dependent on each other.",
    "uncertainty": 8,
    "isDependency": False
    }
]


dominant_response = get_dominat_response(responses=responses)

print(dominant_response)


{'plan': "1. Identify the configuration options in question: both are 'project.modelVersion' of type VERSION_NUMBER with value 4.0.0.\n2. Check if these configurations belong to different Maven modules (apollo-audit/apollo-audit-api and apollo-biz).\n3. Determine if there's a parent-child relationship between the POM files or if they are independent.\n4. Assess whether Maven requires modelVersion to be consistent across modules in a multi-module project.\n5. Conclude based on Maven's requirements for modelVersion consistency.", 'rationale': "In Maven, each module's pom.xml must have the same modelVersion as specified in the parent POM if it's part of a multi-module project. If these modules share a common parent with a defined modelVersion, then their individual 'project.modelVersion' values must match to maintain consistency and avoid configuration errors.", 'uncertainty': 8, 'isDependency': True}
